In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
 
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
 
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
 
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
    
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# 토큰으로 분리된 단어들 중 명사만 추출

In [2]:
# 명사 파일 로드

data=pd.read_excel('D:\\toy\\data\\data_pos_nva.xlsx')

In [3]:
# 컬럼확인

data.columns

Index(['c_n_pos_a', 'c_n_pos_v', 'c_n_pos_n_more1', 'c_n_pos_n_more2',
       't_n_pos_a', 't_n_pos_v'],
      dtype='object')

In [4]:
# 빈도 수 2개 이하는 제거한 명사만 추출

data_n=data['c_n_pos_n_more2']

In [5]:
# 결측치 제거

data_n=data_n.dropna(axis=0)

In [6]:
# 확인

data_n

0         반도체
1          전자
2          기업
3          시장
4          사업
         ... 
46236     중재위
46237      광초
46238     사할린
46239    간접흡연
46240      필순
Name: c_n_pos_n_more2, Length: 46241, dtype: object

In [7]:
# 리스트로 변환

data_nl=data_n.tolist()

In [8]:
# 확인

data_nl

['반도체',
 '전자',
 '기업',
 '시장',
 '사업',
 '기술',
 '삼성',
 '투자',
 '산업',
 '중국',
 '수출',
 '업체',
 '실적',
 '개발',
 '증가',
 '한국',
 '생산',
 '이익',
 '영업',
 '지난해',
 '제품',
 '매출',
 '세계',
 '올해',
 '전망',
 '미국',
 '대비',
 '기록',
 '성장',
 '지난',
 '국내',
 '감소',
 '분야',
 '증권',
 '규모',
 '하락',
 '부문',
 '상승',
 '대한',
 '그룹',
 '자동차',
 '최근',
 '통해',
 '확대',
 '위해',
 '때문',
 '디스플레이',
 '관련',
 '예상',
 '글로벌',
 '회사',
 '주가',
 '장비',
 '스마트폰',
 '이번',
 '이후',
 '계획',
 '정부',
 '개선',
 '경제',
 '공장',
 '회장',
 '현재',
 '이상',
 '연구원',
 '대표',
 '발표',
 '하이닉스',
 '업종',
 '수준',
 '지원',
 '업계',
 '최대',
 '부품',
 '분석',
 '가격',
 '시스템',
 '기자',
 '공급',
 '전략',
 '수요',
 '대해',
 '경우',
 '종목',
 '바이오',
 '예정',
 '일본',
 '설명',
 '판매',
 '상황',
 '단지',
 '가장',
 '인수',
 '달러',
 '기준',
 '지수',
 '공정',
 '우리',
 '거래',
 '지속',
 '메모리',
 '지역',
 '서비스',
 '라며',
 '경쟁력',
 '모바일',
 '갤럭시',
 '목표',
 '협력',
 '영향',
 '미래',
 '평가',
 '매수',
 '사장',
 '관계자',
 '기존',
 '서울',
 '전체',
 '각각',
 '건설',
 '평택',
 '진행',
 '유지',
 '중심',
 '소재',
 '해외',
 '연구',
 '시작',
 '혁신',
 '효과',
 '센터',
 '가능성',
 '문제',
 '주요',
 '사용',
 '나노',
 '확보',
 '경기',
 '경쟁',
 '의견',

In [9]:
# 뉴스 파일 로드

data_all=pd.read_excel('D:\\toy\\data\\online_news_nosw.xlsx')

In [10]:
data_all['Index']

0        198930
1        198929
2        198928
3        198926
4        198927
          ...  
86377    112553
86378    112552
86379    112551
86380    112550
86381    112549
Name: Index, Length: 86382, dtype: int64

In [11]:
data_all.columns

Index(['Index', 'content', 'created_date', 'publisher', 'section', 'title',
       '분기', '실질GDP', 'c_pos', 't_pos', 'c_n_pos', 't_n_pos'],
      dtype='object')

In [12]:
data_all['c_n_pos'][1]

"['냈다', '우세하다', '넘어', '따르면', '아니다', '업계', '효율', '된다', '수익', '격차', '조사', '연간', '일부', '개선', '머물러', '되면', '큼지막한', '센서', '생산', '부진', '위용', '전자공학', '극심한', '고속도로', '아니라', '연구개발', '사업', '예상', '증권', '수주', '처음', '악화', '축구장', '빠졌던', '빠졌다', '외부', '예정', '구매', '이를', '확대', '불과하다', '없어', '준다', '착공', '내다봤다', '양산', '당분간', '왔다', '나온다', '맡겼던', '사상', '체인지', '조용히', '브랜드', '전문', '설계', '전반', '점유', '나올', '해도', '있는', '성장할', '애플', '엑시노스', '수정', '실적', '전자기기', '전기', '차례', '수십', '잡았던', '유일하게', '하는', '사례', '상징', '수출', '핀펫', '자체', '추격', '메모리', '하락', '보이', '퀄컴', '하게', '인쇄공', '앞선', '분야', '최대', '거두는', '치를', '않다', '넘을', '차원', '대비', '솟아', '건설', '라면', '주력', '환하다', '회사', '대표', '독식', '보인다', '가장', '물량', '였다고', '높인', '크기', '역량', '가고', '가격', '하지', '모바일', '이뤄져', '격화', '크레인', '압도', '기술', '해외', '개발', '넘는', '나노미터', '시스템', '되찾아', '끊임없는', '밀려', '규모', '한층', '투자', '수준', '나선', '육박', '조짐', '이어', '느냐가', '신규', '높아질', '시장', '탑재', '국내외', '영업', '대박', '효자', '기업', '반도체', '증설', '가까이', '좁혀', '주자', '명운', '어려울', '달러', '세계', '유회', '직전', '늘어난', '계속'

In [13]:
# n_pos_t 리스트안의 문자형 분리 & 특수문자 제거

data_sp=[]
for i in range(len(data_all)):
    data_sp.append(data_all['c_n_pos'][i].split(','))
    
data_sp[0]

["['부문'",
 " '뉴스'",
 " '따르면'",
 " '떨어질'",
 " '된다'",
 " '연간'",
 " '종합'",
 " '되면'",
 " '기조'",
 " '종목'",
 " '넘길'",
 " '단가'",
 " '증권'",
 " '넘긴'",
 " '올리며'",
 " '확대'",
 " '앞서'",
 " '체인지'",
 " '대한'",
 " '있는'",
 " '성장할'",
 " '맞을'",
 " '하는'",
 " '크게'",
 " '전체'",
 " '있기'",
 " '지를'",
 " '최대'",
 " '같은'",
 " '재다'",
 " '대비'",
 " '때문'",
 " '판매'",
 " '보다'",
 " '이유'",
 " '겪은'",
 " '맛있는'",
 " '보인다'",
 " '기를'",
 " '나왔다'",
 " '서다'",
 " '성장'",
 " '모바일'",
 " '쳐지고'",
 " '이었던'",
 " '투자'",
 " '보이는데도'",
 " '포인트'",
 " '이어'",
 " '시장'",
 " '영업'",
 " '기업'",
 " '반도체'",
 " '걸까'",
 " '세계'",
 " '전자상거래'",
 " '조정'",
 " '했다'",
 " '쳐졌다'",
 " '상향'",
 " '예측'",
 " '전망'",
 " '차지'",
 " '바뀌는'",
 " '정보'",
 " '신선한'",
 " '국내'",
 " '성장한'",
 " '치가'",
 " '수요'",
 " '매출'",
 " '새해'",
 " '분석'",
 " '이익'",
 " '호황'",
 " '사이트'",
 " '역성'",
 " '됐다'",
 " '훌쩍'",
 " '갔다'",
 " '비중'",
 " '신문']"]

In [14]:
data_st=[]
for i in range(len(data_sp)):
    data_st2=[]
    for j in data_sp[i]:
        data_st2.append(j.lstrip('[').rstrip('"]').strip(" ' "))
    data_st.append(data_st2)

data_st[0][1]

'뉴스'

In [15]:
# data_nl을 세트로 변환

a=set(data_nl)

In [16]:
# 명사파일로 필터링하여 n_pos_c 컬럼의 명사만 추출

SW_del=[]
for i in data_st:
    b=set(i)
    SW_del.append(b&a)

In [17]:
# 확인

SW_del

[{'국내',
  '기업',
  '기조',
  '뉴스',
  '단가',
  '대비',
  '대한',
  '때문',
  '매출',
  '모바일',
  '반도체',
  '부문',
  '분석',
  '비중',
  '사이트',
  '상향',
  '새해',
  '성장',
  '세계',
  '수요',
  '시장',
  '신문',
  '앞서',
  '역성',
  '연간',
  '영업',
  '예측',
  '이유',
  '이익',
  '전망',
  '전자상거래',
  '전체',
  '정보',
  '조정',
  '종목',
  '종합',
  '증권',
  '차지',
  '체인지',
  '최대',
  '치가',
  '크게',
  '투자',
  '판매',
  '포인트',
  '호황',
  '확대',
  '훌쩍'},
 {'가격',
  '가까이',
  '가장',
  '개발',
  '개선',
  '거래',
  '건설',
  '격차',
  '격화',
  '경기',
  '경쟁',
  '계속',
  '계획',
  '고속도로',
  '공략',
  '공장',
  '공정',
  '과언',
  '과학기술원',
  '관계자',
  '구매',
  '국내외',
  '규모',
  '기술',
  '기업',
  '나노미터',
  '나선',
  '노릇',
  '달러',
  '당분간',
  '대박',
  '대비',
  '대표',
  '덕분',
  '독식',
  '라면',
  '로고',
  '메모리',
  '명운',
  '모바일',
  '물량',
  '미세',
  '반도체',
  '보고',
  '보이',
  '본격',
  '부진',
  '부착',
  '분석',
  '분야',
  '브랜드',
  '비메',
  '사례',
  '사상',
  '사업',
  '사이트',
  '산업',
  '상징',
  '상향',
  '새해',
  '생산',
  '선두',
  '설계',
  '설비',
  '세계',
  '센서',
  '수십',
  '수익',
  '수정',
  '수주',
  '수준',
  '수출',
  '스마트폰',
  '시스

In [18]:
# set을 리스트로 변환

SW_del2=[]
for i in SW_del:
    SW_del2.append(list(i))

In [19]:
SW_del2[55000]

['확대',
 '수요',
 '물량',
 '기준',
 '광학',
 '차지',
 '금융투자',
 '반도체',
 '고영은',
 '압도',
 '최초',
 '매수',
 '포인트',
 '판매',
 '개발',
 '성장',
 '의미',
 '지속',
 '기능',
 '사업',
 '동사',
 '바탕',
 '감소',
 '안정',
 '증가',
 '목표',
 '제시',
 '머니투데이',
 '경영',
 '자본',
 '커버',
 '전망',
 '의견',
 '베스트',
 '전장',
 '선정',
 '이익',
 '제품',
 '리포트',
 '현금',
 '초도',
 '신뢰',
 '반영',
 '증권',
 '추진',
 '세계',
 '대비',
 '경쟁',
 '요약',
 '시장',
 '독보',
 '연간',
 '단가',
 '평가',
 '계속',
 '기업',
 '주가',
 '개시',
 '의료',
 '고객',
 '불구',
 '위해',
 '내비게이션',
 '영업',
 '내용',
 '매출',
 '경우',
 '주력',
 '검사',
 '글로벌',
 '통해',
 '실적',
 '예상',
 '고가',
 '부가',
 '여개',
 '보고서',
 '영향',
 '하이엔드',
 '투자',
 '상승',
 '기술',
 '시장점유율',
 '중이',
 '공정',
 '추가',
 '강소기업',
 '장비',
 '전세계',
 '상태',
 '규모',
 '자동',
 '불량품',
 '여력',
 '차입',
 '납품',
 '출하',
 '측면',
 '로서',
 '겸비',
 '달러',
 '대해',
 '가시',
 '성과',
 '기대',
 '뇌수술',
 '자동차']